# MNIST Classification with JaxFlow
This notebook demonstrates how to build and train a ResNet CNN Neural Network on the MNIST dataset using JaxFlow.

In [ ]:
# if you want to use jaxflow for cpu, please install jaxflow with CPU support
! pip install --upgrade jaxflow
# if you want to use GPU, please install jaxflow with GPU support
# ! pip install --upgrade jaxflow[gpu]
# if you want to use TPU, please install jaxflow with TPU support
# ! pip install --upgrade jaxflow[tpu]


In [ ]:
import jax
import jax.numpy as jnp
import jaxflow as jf
from jaxflow.models import Model
from jaxflow.layers import Conv2D, MaxPooling2D, Dense, Flatten
from jaxflow.initializers import GlorotUniform, Zeros
from jaxflow.optimizers import Adam
from jaxflow.losses import SparseCategoricalCrossentropy
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

## 1. Load and preprocess MNIST
We first load the MNIST dataset and normalize pixel values to the [0, 1] range. We also add a channel dimension for compatibility with Conv2D layers.

In [ ]:
# Load and preprocess MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(jnp.float32) / 255.0
x_test = x_test.astype(jnp.float32) / 255.0
x_train = x_train[..., None]
x_test = x_test[..., None]
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

## 2. Define the CNN model
We define a  ResNet Block using jf.layers.Layer

In [ ]:

class ResNetBlock(jf.layers.Layer):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        # First conv + BN + ReLU
        self.conv1 = Conv2D(filters=out_channels,
                            kernel_size=(3,3),
                            strides=(stride,stride),
                            padding='SAME',
                            use_bias=True)

        # Second conv + BN (no ReLU here)
        self.conv2 = Conv2D(filters=out_channels,
                            kernel_size=(3,3),
                            strides=(1,1),
                            padding='SAME',
                            use_bias=True)


    def build(self, input_shape):
        # Build conv1 on the true input...
        self.conv1.build(input_shape)
        self.conv2.build([input_shape[0], input_shape[1], input_shape[2], self.conv1.filters])




    def call(self, x, training=False, mask=None):
        identity = x

        out = self.conv1(x, training=training)
        out = jf.activations.relu(out)

        out = self.conv2(out, training=training)
        out += identity
        return out


###  Subclassing Model Building 


In [ ]:
class CNN(Model):
    def __init__(self, num_classes: int = 10, name: str = "MyCNN"):
        super().__init__(name=name)
        self.conv1 = Conv2D(filters=64, kernel_size=(3,3), activation=jf.activations.relu, kernel_initializer=GlorotUniform, bias_initializer=Zeros)
        self.pool1 = MaxPooling2D(pool_size=(2,2))
        self.ln1 = jf.layers.LayerNormalization()
        self.conv2 = ResNetBlock(in_channels=64, out_channels=64)
        self.pool2 = MaxPooling2D(pool_size=(2,2))
        self.flatten = jf.layers.GlobalAveragePooling2D()
        self.ln2 = jf.layers.LayerNormalization()
        self.dense1 = Dense(units=64, activation=jf.activations.relu, kernel_initializer=GlorotUniform, use_bias=False)
        self.outputs = Dense(units=num_classes, activation=jf.activations.softmax, kernel_initializer=GlorotUniform, bias_initializer=Zeros)
    def call(self, inputs, training: bool = False):
        x = self.conv1(inputs, training=training)
        x = self.pool1(x, training=training)
        x = self.ln1(x, training=training)
        x = self.conv2(x, training=training)
        x = self.pool2(x, training=training)
        x = self.flatten(x)
        x = self.ln2(x, training=training)
        x = self.dense1(x, training=training)
        return self.outputs(x, training=training)

# Build the model
model = CNN(num_classes=10)
model.build(input_shape=(None, 28, 28, 1))
print(model.summary())

### OR Using the `.add()` Method (Sequential-style API)

In [ ]:
model = Model()
model.add(Conv2D(filters=64, kernel_size=(3,3), activation=jf.activations.relu, kernel_initializer=GlorotUniform, bias_initializer=Zeros,
               padding='SAME',))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(jf.layers.LayerNormalization())
model.add(ResNetBlock(in_channels=64, out_channels=64))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(jf.layers.GlobalAveragePooling2D())
model.add(jf.layers.LayerNormalization())
model.add(Dense(units=64, activation=jf.activations.relu, kernel_initializer=GlorotUniform, use_bias=False))
model.add(Dense(units=10, activation=jf.activations.softmax, kernel_initializer=GlorotUniform, bias_initializer=Zeros))
model.build(input_shape=(None, 28, 28, 1))
print(model.summary())

## 3. Compile and train the model
We use the Adam optimizer and sparse categorical crossentropy loss. We train for 5 epochs with a batch size of 128.

In [ ]:
optimizer = Adam(learning_rate=0.001)
loss_fn = SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss_fn=loss_fn,)
history = model.fit(x_train, y_train, epochs=510, batch_size=128, validation_data=(x_test, y_test), verbose=2)

## 4. Evaluate on the test set

In [ ]:
test_loss = model.evaluate(x_test, y_test, batch_size=128)
print(f"Test Loss: {test_loss}")

In [ ]:
pred = model.predict(x_test)
pred = jnp.argmax(pred, -1)


In [ ]:
accuracy = jf.metrics.accuracy(y_test, pred)
precision = jf.metrics.precision(y_test, pred, average='macro',num_classes=10)
recall = jf.metrics.recall(y_test, pred, average='macro',num_classes=10)
f1 = jf.metrics.f1_score(y_test, pred, average='macro',num_classes=10)
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

## 5. Plot training history

In [ ]:
epochs = range(1, len(history['loss'])+1)
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(epochs, history['loss'], label='Train Loss')
plt.plot(epochs, history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()

### Conclusion
This notebook showed how to train a CNN on MNIST with JaxFlow. Feel free to experiment with different architectures, learning rates, and batch sizes to improve performance!